<a href="https://colab.research.google.com/github/yazhcodes/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installation and Setup**

In [ ]:
# Set Spark Version
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Java, Spark and PostgreSQL Driver
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Store DB Server Password
from getpass import getpass
password = getpass('Enter DB Server password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cmvclswi6ved.us-east-2.rds.amazonaws.com:5432/AmazonVineDB"
config = {"user":"postgres",
          "password": password, # Canada2021
          "driver":"org.postgresql.Driver"}

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [61.8 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [ ]:
# Import Dependencies
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_date

# **Extraction**

In [ ]:
# Read Raw Data
url ="https://yazhcodes-amozon-vine.s3.us-east-2.amazonaws.com/amazon_reviews_us_Home_Improvement_v1_00.tsv"
spark.sparkContext.addFile(url)
raw_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv"), sep="\t", header=True)
raw_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

# **Transformation**

In [ ]:
raw_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
type_cast_df = raw_df.withColumn('customer_id',raw_df['customer_id'].cast(IntegerType()))
type_cast_df = type_cast_df.withColumn('product_parent',type_cast_df['product_parent'].cast(IntegerType()))
type_cast_df = type_cast_df.withColumn('star_rating',type_cast_df['star_rating'].cast(IntegerType()))
type_cast_df = type_cast_df.withColumn('helpful_votes',type_cast_df['helpful_votes'].cast(IntegerType()))
type_cast_df = type_cast_df.withColumn('total_votes',type_cast_df['total_votes'].cast(IntegerType()))
type_cast_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Reviews Dataframe
review_id_df = type_cast_df.select([
                                    'review_id',
                                    'customer_id',
                                    'product_id',
                                    'product_parent',
                                    to_date('review_date','yyyy-MM-dd').alias('review_date')
                                    ])
review_id_df.show()
review_id_df.printSchema()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R215C9BDXTDQOW|   48881148|B00FR4YQYK|     381800308| 2015-08-31|
|R1DTPUV1J57YHA|   47882936|B00439MYYE|     921341748| 2015-08-31|
| RFAZK5EWKJWOU|   44435471|B00002N762|      56053291| 2015-08-31|
|R2XT8X0O0WS1AL|   28377689|B000QFCP1G|     595928517| 2015-08-31|
|R14GRNANKO2Y2J|   50134766|B00WRCRKOI|     417053744| 2015-08-31|
|R2BLF9VYL24LCQ|   14066511|B00NIH88EW|     275395071| 2015-08-31|
|R1GI9UW5KJ671O|   15211046|B005B9CI96|     856617815| 2015-08-31|
|R2H5CEJN863M86|   14862498|B008L0OMWI|     125102494| 2015-08-31|
| R5PPDHFOZ3SMU|   23617292|B00P9FTC6O|     523110842| 2015-08-31|
| RE1L9IENKJJ7Y|   35820485|B00K6BQEHQ|     797306964| 2015-08-31|
|R3CZ0990QC2Z0H|   47162350|B008BYQCWM|     865874404| 2015-08-31|
|R3UMMD2IO29QSP|   31884789|B003BLHTOU|     183592595| 2015-08

In [ ]:
# Products Dataframe
products_df = type_cast_df.select([
                                   'product_id',
                                   'product_title'
                                   ])
products_df = products_df.dropDuplicates(subset=["product_id"])
products_df.show()
products_df.printSchema()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|7800807606|CBconcept Brand H...|
|9178907519|IKEA - GAVIK Tabl...|
|B00002N70K|Kwikset 293 Small...|
|B00002N7BC|Westinghouse 6720...|
|B00002NAEP|Westinghouse One-...|
|B00002NAVH|Leviton L02-49686...|
|B00002NB2F|View-Pak UBIP2020...|
|B00004SQLR|Master Lock 1547D...|
|B00004WLKJ|Gardner Bender He...|
|B00004Y61P|Towing/Recovery S...|
|B00004YUN1|Leviton 6697-W An...|
|B00004Z4AV|3M 4.875 by 2.875...|
|B0000AHSA7|Pollenex DP1011 H...|
|B0000BYEF5|CONROS #PM00506 6...|
|B0000BYEF6|Lutron Electronic...|
|B0000CEZ7C|Northern Industri...|
|B0000DBIKH|Brita 42633 Fauce...|
|B0000DD0F2|Shop-Vac 905-03-0...|
|B0000DEZNE|Dale Adams Enterp...|
|B0000DI5II|GE 75 Watt 2 Pack...|
+----------+--------------------+
only showing top 20 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# Customers Dataframe
customers_df = type_cast_df.groupBy('customer_id').agg({'customer_id':'count'})
customers_df = customers_df.withColumnRenamed('count(customer_id)','customer_count')
customers_df.show()
customers_df.printSchema()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28377689|             1|
|   28258386|             2|
|    9967574|             1|
|   25153155|             1|
|   10088068|             1|
|   45657423|             1|
|   19021463|             1|
|   41413793|             2|
|   43789873|             2|
|   12406466|             1|
|   41045019|             1|
|   35535911|             2|
|   47108763|             1|
|   48113150|             1|
|   18201417|             9|
|   26079415|             3|
|   36114891|             7|
|    3712628|             1|
|   37499901|             2|
|   47321438|             2|
+-----------+--------------+
only showing top 20 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
# Vine Dataframe
vine_df = type_cast_df.select([
                               'review_id',
                               'star_rating',
                               'helpful_votes',
                               'total_votes',
                               'vine',
                               'verified_purchase'
                               ])
vine_df.show()
vine_df.printSchema()

# **Load**

In [ ]:
# Write DataFrame to tables in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)